## RandomForestによる、切り出し済み画像に対する分類

In [20]:
from pathlib import Path
from typing import cast

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def load_dataset(data_dir: Path) -> tuple[list[np.ndarray], np.ndarray, list[str]]:
    """
    データセットを読み込む
    
    Args:
        data_dir: データセットのディレクトリ
    
    Returns:
        images: 画像のリスト（各画像はサイズが異なる可能性あり）
        labels: ラベル配列
        class_names: クラス名のリスト
    """
    class_names = ["graupel", "snowflake"]
    images, labels = [], []

    for label, class_name in enumerate(class_names):
        for img_path in sorted(data_dir.glob(f"{class_name}_*.png")):
            img = cv2.imread(str(img_path))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                images.append(img)
                labels.append(label)

    return images, np.array(labels), class_names


def extract_features(image: np.ndarray) -> np.ndarray:
    """
    1枚の画像から特徴量を抽出する

    Args:
        image: RGB画像 (H, W, 3)

    Returns:
        特徴量ベクトル (1次元配列)
    """
    features = []

    # TODO: カラー画像をグレースケールに変換 --> 二値化して輪郭を抽出


    if len(contours) == 0:
        # 輪郭が見つからない場合はゼロで埋める
        features.append(0)
    else:
        # TODO: 特徴量を作成

    return np.array(features)


def run_cross_validation(
    images: list[np.ndarray],
    labels: np.ndarray,
    n_folds: int = 5,
    random_seed: int = 42,
) -> tuple[list[int], list[int]]:
    """
    クロスバリデーションを実行する

    Args:
        images: 画像データのリスト（各画像は(H, W, C)のRGB画像）
        labels: ラベル配列 (N,)
        n_folds: 分割数
        random_seed: 乱数シード

    Returns:
        all_preds: 全Foldの予測結果
        all_labels: 全Foldのラベル
    """
    # StratifiedKFoldでデータを分割
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)

    # 全Foldの予測結果を保存するリスト
    all_preds = []
    all_labels = []

    # TODO:各Foldで訓練と評価を実行

    return all_preds, all_labels


def plot_confusion_matrix(
    confusion_matrix: np.ndarray,
    class_names: list[str],
    output_path: Path,
) -> None:
    """混同行列をプロットする"""
    fig, ax = plt.subplots(figsize=(6, 5))
    cm: np.ndarray = cast(np.ndarray, confusion_matrix)
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues",
        xticklabels=class_names, yticklabels=class_names, ax=ax
    )
    ax.set_title(f"Confusion Matrix")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()
    print(f"Saved: {output_path}")


def main(
    data_dir: Path,
    output_dir: Path,
    n_folds: int,
    seed: int,
) -> None:
    """メイン関数"""
    output_dir.mkdir(parents=True, exist_ok=True)

    # データ読み込み
    images, labels, class_names = load_dataset(data_dir)
    print(f"Samples: {len(images)}, Classes: {dict(zip(class_names, np.bincount(labels)))}")

    # クロスバリデーション
    print(f"\nRunning {n_folds}-fold cross validation...")
    all_preds, all_labels = run_cross_validation(images, labels, n_folds, seed)

    # 混同行列を作成
    matrix = confusion_matrix(all_labels, all_preds)

    # 混同行列を保存
    print("\nSaving confusion matrix...")
    plot_confusion_matrix(matrix, class_names, output_dir / "confusion_matrix.png")

    print("\nDone!")


main(
    data_dir=Path("../dataset/"),
    output_dir=Path("../output/"),
    n_folds=5,
    seed=42
)